In [20]:
import sys
import platform
sys.path.insert(0, '../')
%load_ext autoreload
%autoreload 2
import os
from lib.utils import check_exists, run_python, run_matlab, solver_name_to_n_iter
import itertools

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [30]:
percNonUVal_list = [95]
budget_list = [10000]
VOT_list = [2.63] 
solver_list = ["Gurobi", "Mosek", "ADMM"] # ADMM is Algorithm 1
config_filename = "../data/YAML/region_toy.yaml"
nPath = 2
n_iter_UE = 5
# percNonUVal = 50
n_time = 204
n_time_inc_start = 13
n_time_inc_end = 24
step_size_UE = 0.01
region_ = "region_toy"
setting_region = "5_22_AVG5_th1_pad_MultipleTimes"
fileName0 = "_StartHour_5_AVG5_pad_theta1e+00"
folderName0 = "Mar2May_2018_new_5-22_"
fairness = "0_0_0_100_0" # Corresponds to RHSMultiplier = [1, 1.1, 1.5, 2, 2.5]
n_iter_ADMM = 100
seed_ADMM = 2
seed_solver = 2
seed_data = 2
solve_binarization = 1 # For solvers other than Algorithm 1: 1=True, 0=False
n_companies_ADMM = 1
rho = 20
MIPGap = 0.01

In [26]:
# n_companies_cost_list = [1, 10, 100, 1000] # Number of companies in cost computation 
min_n_companies = 1 # The min number of organizations in the cost computation
max_n_companies = 10000 # The max number of organizations in the cost computation
# We compute the cost of incentivization in two settings. 
# Setting 1: number of organization is:
# range(min_n_companies, step_n_companies, max_n_companies) + {# of drivers OR individual drivers}
# Setting 1 is set if factor_n_companies=0. Setting 1 is much more time consuming but more details comapred to setting 2.
step_n_companies = 10 
# Setting 2: number of organization is:
# {min_n_companies, min_n_companies*factor_n_companies, ..., max_n_companies, # of drivers OR individual drivers}
# If factor_n_companies=-1, 1,2,3, and 5 companies will be created
factor_n_companies = 10
# Number of random sampling of drivers to companies and computing cost of incentivization
n_sample = 10

## create_graph_py2 (Py2)

In [11]:
run_python(py_version = 2, 
           py_file = "../data_creator/createGraphPy2.py", 
           arg_names = ["config_filename", "nPath"], 
           arg_values = [config_filename, nPath])

Running command:  python2 ../data_creator/createGraphPy2.py --config_filename ../data/YAML/region_toy.yaml --nPath 2


## create_graph_py3 (Py3)

In [12]:
run_python(py_version = 3, 
           py_file = "../data_creator/createGraphPy3.py", 
           arg_names = ["config_filename", "nPath"], 
           arg_values = [config_filename, nPath])

Running command:  python3 ../data_creator/createGraphPy3.py --config_filename ../data/YAML/region_toy.yaml --nPath 2


## DataCreator1 (Py3)

In [13]:
run_python(py_version = 3, 
    py_file = "../data_creator/DataCreator1.py", 
    arg_names = ["config_filename"], 
    arg_values = [config_filename],
    )

Running command:  python3 ../data_creator/DataCreator1.py --config_filename ../data/YAML/region_toy.yaml


## HistoricalData (Py2)

In [14]:
run_python(py_version = 2, 
    py_file = "../data_creator/HistoricalData.py", 
    arg_names = ["config_filename"], 
    arg_values = [config_filename],
    )

Running command:  python2 ../data_creator/HistoricalData.py --config_filename ../data/YAML/region_toy.yaml


## 1. DataLoader_Det (MATLAB) 2. runDet (Py2) 3. DataCreator2 (Py3)

In [15]:
for i in range(n_iter_UE):
    DataLoader_Det_input_list = [i, \
                                 n_time, \
                                 n_time_inc_start, \
                                 n_time_inc_end, \
                                 seed_data, \
                                 step_size_UE, \
                                 region_, \
                                 setting_region, \
                                 fileName0, \
                                 folderName0]
    run_matlab(matlab_function = "addpath('../data_creator/');DataLoader_Det", 
                matlab_input_list = DataLoader_Det_input_list, 
                nojvm = False,
              )
    
    run_python(py_version = 2, 
               py_file = "../data_creator/runDet.py", 
               arg_names = ["config_filename", \
                            "sD", \
                            "iterRun", \
                            "initEta"], 
               arg_values = [config_filename, \
                             seed_data, \
                             i, \
                             -1],
              )
    
    run_python(py_version = 3, 
               py_file = "../data_creator/DataCreator2.py", 
               arg_names = ["config_filename", \
                            "sD", \
                            "iterRun", \
                            "initEta"], 
               arg_values = [config_filename, \
                             seed_data, \
                             i, \
                             -1])

Running command:  try;addpath('../data_creator/');DataLoader_Det(0,204,13,24,2,0.01,'region_toy','5_22_AVG5_th1_pad_MultipleTimes','_StartHour_5_AVG5_pad_theta1e+00','Mar2May_2018_new_5-22_');catch;end;quit
Running command:  python2 ../data_creator/runDet.py --config_filename ../data/YAML/region_toy.yaml --sD 2 --iterRun 0 --initEta -1
Running command:  python3 ../data_creator/DataCreator2.py --config_filename ../data/YAML/region_toy.yaml --sD 2 --iterRun 0 --initEta -1
Running command:  try;addpath('../data_creator/');DataLoader_Det(1,204,13,24,2,0.01,'region_toy','5_22_AVG5_th1_pad_MultipleTimes','_StartHour_5_AVG5_pad_theta1e+00','Mar2May_2018_new_5-22_');catch;end;quit
Running command:  python2 ../data_creator/runDet.py --config_filename ../data/YAML/region_toy.yaml --sD 2 --iterRun 1 --initEta -1
Running command:  python3 ../data_creator/DataCreator2.py --config_filename ../data/YAML/region_toy.yaml --sD 2 --iterRun 1 --initEta -1
Running command:  try;addpath('../data_creator/');

## initEta_prestep (MATLAB)

In [16]:
# (n_companies, iterRun, nonuser_perc_prob0, n_time, n_time_inc_start, n_time_inc_end, ...
#     step_size, region_, setting_region, fileName0, folderName0)
initEta_prestep_input_list = [n_iter_UE, \
                              n_time, \
                              n_time_inc_start, \
                              n_time_inc_end, \
                              seed_data, \
                              step_size_UE, \
                              region_, \
                              setting_region, \
                              fileName0, \
                              folderName0]
run_matlab(matlab_function = "addpath('../data_creator/');initEta_prestep", 
            matlab_input_list = initEta_prestep_input_list, 
            nojvm = False,
        )

Running command:  try;addpath('../data_creator/');initEta_prestep(5,204,13,24,2,0.01,'region_toy','5_22_AVG5_th1_pad_MultipleTimes','_StartHour_5_AVG5_pad_theta1e+00','Mar2May_2018_new_5-22_');catch;end;quit


## runDet (Py2)

In [17]:
run_python(py_version = 2, 
           py_file = "../data_creator/runDet.py", 
           arg_names = ["config_filename", \
                        "sD", \
                        "iterRun", \
                        "initEta"], 
           arg_values = [config_filename, \
                         seed_data, \
                         n_iter_UE, \
                         1],
          )

Running command:  python2 ../data_creator/runDet.py --config_filename ../data/YAML/region_toy.yaml --sD 2 --iterRun 5 --initEta 1


## DataCreator2 (Py3)

In [18]:
run_python(py_version = 3, 
           py_file = "../data_creator/DataCreator2.py", 
           arg_names = ["config_filename", \
                        "sD", \
                        "iterRun", \
                        "initEta"], 
           arg_values = [config_filename, \
                         seed_data, \
                         n_iter_UE, \
                         1])

Running command:  python3 ../data_creator/DataCreator2.py --config_filename ../data/YAML/region_toy.yaml --sD 2 --iterRun 5 --initEta 1


## Solving the Problem (MATLAB)

In [32]:
for percNonUVal, VOT, solver_name, budget in itertools.product(percNonUVal_list, VOT_list, solver_list, budget_list):
    n_iter_algo = n_iter_ADMM if solver_name == "ADMM" else solver_name_to_n_iter(solver_name)
    if solver_name == "ADMM":
        # Run Algorithm 1
        ADMM_input_list = [n_iter_UE, \
                           percNonUVal, \
                           budget, \
                           n_iter_algo, \
                           n_time, \
                           n_time_inc_start, \
                           n_time_inc_end, \
                           VOT, \
                           seed_ADMM, \
                           seed_data, \
                           rho, \
                           fairness, \
                           step_size_UE, \
                           region_, \
                           setting_region, \
                           fileName0, \
                           folderName0, \
                          ]
        run_matlab(matlab_function = "addpath('../incentivization/');ADMM", 
                    matlab_input_list = ADMM_input_list, 
                    nojvm = True,
            )

        # Find binary solution to the provided solution by Algorithm 1 
        ILP_input_list = [n_iter_UE, \
                           percNonUVal, \
                           budget, \
                           n_iter_algo, \
                           n_time, \
                           n_time_inc_start, \
                           n_time_inc_end, \
                           VOT, \
                           seed_ADMM, \
                           seed_data, \
                           rho, \
                           fairness, \
                           step_size_UE, \
                           region_, \
                           setting_region, \
                           MIPGap, \
                          ]
        run_matlab(matlab_function = "addpath('../incentivization/');ILP", 
                    matlab_input_list = ILP_input_list, 
                    nojvm = True,
            )
    else:
        # Solve the MIP optimization problem directly using a solver
        solver_input_list = [solver_name, \
                             MIPGap, \
                             solve_binarization, \
                               n_time, \
                               n_time_inc_start, \
                               n_time_inc_end, \
                               seed_data, \
                               seed_solver, \
                               percNonUVal, \
                               budget, \
                               VOT, \
                               fairness, \
                               n_iter_UE, \
                               step_size_UE, \
                               region_, \
                               setting_region, \
                               fileName0, \
                               folderName0]
        run_matlab(matlab_function = "addpath('../incentivization/');solver_incentivization", 
                    matlab_input_list = solver_input_list, 
                    nojvm = True,
                )


Running command:  try;addpath('../incentivization/');solver_incentivization('Gurobi',0.01,1,204,13,24,2,2,95,10000,2.63,'0_0_0_100_0',5,0.01,'region_toy','5_22_AVG5_th1_pad_MultipleTimes','_StartHour_5_AVG5_pad_theta1e+00','Mar2May_2018_new_5-22_');catch;end;quit
Running command:  try;addpath('../incentivization/');solver_incentivization('Mosek',0.01,1,204,13,24,2,2,95,10000,2.63,'0_0_0_100_0',5,0.01,'region_toy','5_22_AVG5_th1_pad_MultipleTimes','_StartHour_5_AVG5_pad_theta1e+00','Mar2May_2018_new_5-22_');catch;end;quit
Running command:  try;addpath('../incentivization/');ADMM(5,95,10000,100,204,13,24,2.63,2,2,20,'0_0_0_100_0',0.01,'region_toy','5_22_AVG5_th1_pad_MultipleTimes','_StartHour_5_AVG5_pad_theta1e+00','Mar2May_2018_new_5-22_');catch;end;quit
Running command:  try;addpath('../incentivization/');ILP(5,95,10000,100,204,13,24,2.63,2,2,20,'0_0_0_100_0',0.01,'region_toy','5_22_AVG5_th1_pad_MultipleTimes',0.01);catch;end;quit


## getSV_realCost (MATLAB)

In [33]:
for percNonUVal, VOT, solver_name, budget in itertools.product(percNonUVal_list, VOT_list, solver_list, budget_list):
    n_iter_algo = n_iter_ADMM if solver_name == "ADMM" else solver_name_to_n_iter(solver_name)
    # Get the volume and speed based on the post-incentivization traffic
    getSV_realCost_input_list = [n_iter_UE, \
                                 percNonUVal, \
                                 budget, \
                                 n_companies_ADMM, \
                                 n_iter_algo, \
                                 n_time, \
                                 n_time_inc_start, \
                                 n_time_inc_end, \
                                 fairness, \
                                 VOT, \
                                 seed_ADMM, \
                                 seed_data, \
                                 rho, \
                                 step_size_UE,\
                                 region_, \
                                 setting_region, \
                                 MIPGap, ]
    run_matlab(matlab_function = "addpath('../incentivization/');getSV_realCost", 
                matlab_input_list = getSV_realCost_input_list, 
                nojvm = False,
            )

Running command:  try;addpath('../incentivization/');getSV_realCost(5,95,10000,1,-1,204,13,24,'0_0_0_100_0',2.63,2,2,20,0.01,'region_toy','5_22_AVG5_th1_pad_MultipleTimes',0.01);catch;end;quit
Running command:  try;addpath('../incentivization/');getSV_realCost(5,95,10000,1,-2,204,13,24,'0_0_0_100_0',2.63,2,2,20,0.01,'region_toy','5_22_AVG5_th1_pad_MultipleTimes',0.01);catch;end;quit
Running command:  try;addpath('../incentivization/');getSV_realCost(5,95,10000,1,100,204,13,24,'0_0_0_100_0',2.63,2,2,20,0.01,'region_toy','5_22_AVG5_th1_pad_MultipleTimes',0.01);catch;end;quit


## run_realCost (Py2)

In [34]:
for percNonUVal, VOT, solver_name, budget in itertools.product(percNonUVal_list, VOT_list, solver_list, budget_list):
    n_iter_algo = n_iter_ADMM if solver_name == "ADMM" else solver_name_to_n_iter(solver_name)
    # Compute the R matrix and other info based on new speed and volume
    run_python(py_version = 2, 
               py_file = "../incentivization/run_realCost.py", 
               arg_names = ["config_filename", \
                            "sA", \
                            "sD", \
                            "nC", \
                            "f", \
                            "percNonU", \
                            "iterRun", \
                            "b", \
                            "VOT", \
                            "nTIS", \
                            "nTIE", \
                            "it", \
                            "MIPGap", \
                           ], 
               arg_values = [config_filename, \
                             seed_ADMM, \
                             seed_data, \
                             n_companies_ADMM, \
                             fairness, \
                             percNonUVal, \
                             n_iter_UE, \
                             budget, \
                             VOT, \
                             n_time_inc_start, \
                             n_time_inc_end, \
                             n_iter_algo, \
                             MIPGap, \
                            ],
              )

Running command:  python2 ../incentivization/run_realCost.py --config_filename ../data/YAML/region_toy.yaml --sA 2 --sD 2 --nC 1 --f 0_0_0_100_0 --percNonU 95 --iterRun 5 --b 10000 --VOT 2.63 --nTIS 13 --nTIE 24 --it -1 --MIPGap 0.01
Running command:  python2 ../incentivization/run_realCost.py --config_filename ../data/YAML/region_toy.yaml --sA 2 --sD 2 --nC 1 --f 0_0_0_100_0 --percNonU 95 --iterRun 5 --b 10000 --VOT 2.63 --nTIS 13 --nTIE 24 --it -2 --MIPGap 0.01
Running command:  python2 ../incentivization/run_realCost.py --config_filename ../data/YAML/region_toy.yaml --sA 2 --sD 2 --nC 1 --f 0_0_0_100_0 --percNonU 95 --iterRun 5 --b 10000 --VOT 2.63 --nTIS 13 --nTIE 24 --it 100 --MIPGap 0.01


## compareCosts_realCost_initAll2 (MATLAB)

In [35]:
for percNonUVal, VOT, solver_name, budget in itertools.product(percNonUVal_list, VOT_list, solver_list, budget_list):
    n_iter_algo = n_iter_ADMM if solver_name == "ADMM" else solver_name_to_n_iter(solver_name)
    # Compute the incentivization cost
    compareCosts_realCost_initAll2_input_list = [n_iter_UE, \
                                                 percNonUVal, \
                                                 budget, \
                                                 n_companies_ADMM, \
                                                 n_iter_algo, \
                                                 min_n_companies, \
                                                 max_n_companies, \
                                                 step_n_companies, \
                                                 factor_n_companies, \
                                                 # Number of samples of cost computation, randomness is in selecting companies' drivers
                                                 n_sample, \
                                                 n_time, \
                                                 n_time_inc_start, \
                                                 n_time_inc_end, \
                                                 fairness, \
                                                 VOT, \
                                                 seed_ADMM, \
                                                 seed_data, \
                                                 rho, \
                                                 step_size_UE,\
                                                 region_, \
                                                 setting_region, \
                                                 MIPGap, \
                                                ]
    run_matlab(matlab_function = "addpath('../incentivization/');compareCosts_realCost_initAll2_allInOne", 
                matlab_input_list = compareCosts_realCost_initAll2_input_list, 
                nojvm = True,
            )

Running command:  try;addpath('../incentivization/');compareCosts_realCost_initAll2_allInOne(5,95,10000,1,-1,1,10000,10,10,10,204,13,24,'0_0_0_100_0',2.63,2,2,20,0.01,'region_toy','5_22_AVG5_th1_pad_MultipleTimes',0.01);catch;end;quit
Running command:  try;addpath('../incentivization/');compareCosts_realCost_initAll2_allInOne(5,95,10000,1,-2,1,10000,10,10,10,204,13,24,'0_0_0_100_0',2.63,2,2,20,0.01,'region_toy','5_22_AVG5_th1_pad_MultipleTimes',0.01);catch;end;quit
Running command:  try;addpath('../incentivization/');compareCosts_realCost_initAll2_allInOne(5,95,10000,1,100,1,10000,10,10,10,204,13,24,'0_0_0_100_0',2.63,2,2,20,0.01,'region_toy','5_22_AVG5_th1_pad_MultipleTimes',0.01);catch;end;quit
